In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=68649d5c8f35fe6324eb47f65dc7966e50d1e9f278186fbe158c9802d329adb8
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [ ]:
import pandas as pd
import googletrans
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Read the dataset with a specific encoding
input_csv_file = 'spam.csv'  # Replace with the path of your dataset

# Try different encodings if necessary
df = pd.read_csv(input_csv_file, encoding='ISO-8859-1')

# Function to translate text to Telugu
def translate_to_telugu(text):
    try:
        translated = translator.translate(text, src='en', dest='te')
        return translated.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

# Translate only the 'v2' column
df['v2'] = df['v2'].apply(translate_to_telugu)

# Save the updated DataFrame with translated 'v2' column
output_csv_file = 'telugu_translated_dataset.csv'  # Output CSV file with Telugu text in 'v2'
df.to_csv(output_csv_file, index=False, encoding='utf-8')

print(f"CSV file successfully translated and saved as {output_csv_file}")


Error translating text: list index out of range
CSV file successfully translated and saved as telugu_translated_dataset.csv


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with 'v1' as labels and 'v2' as text data
# Replace this with your actual DataFrame load process
# df = pd.read_csv('your_dataset.csv')

# Separate the ham and spam rows
ham_df = df[df['v1'] == 'ham']
spam_df = df[df['v1'] == 'spam']

# Duplicate the spam rows to make their count 1447 (currently 747, so we need 700 more)
spam_duplicated = pd.concat([spam_df] * 2, ignore_index=True)

# Now we have 1447 spam rows
# Randomly sample 700 rows from ham to drop
ham_reduced = ham_df.sample(n=4125, random_state=42)

# Combine the modified ham and spam DataFrames
balanced_df = pd.concat([ham_reduced, spam_duplicated])

# Shuffle the resulting DataFrame to mix the rows
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# If you want to save the resulting balanced DataFrame to a CSV
balanced_df.to_csv('new_dataset.csv', index=False)

# Display the balanced dataset's class distribution
print(balanced_df['v1'].value_counts())


v1
ham     4125
spam    1494
Name: count, dtype: int64


In [ ]:
import pandas as pd
import requests

# ✅ Replace with your actual API key and region
API_KEY = "FrN1fqSKqGS60hCdVaCuy46V63oF7Kp5M8dz3EJ8q3eDrkzG4jVPJQQJ99BBACYeBjFXJ3w3AAAbACOGzZGM"  # <-- Paste your Key 1 here
LOCATION = "eastus"  # Example: "westeurope", "eastus", etc.

# Microsoft Translator API endpoint
ENDPOINT = "https://api.cognitive.microsofttranslator.com/translate"

# ✅ Load your CSV file with a different encoding
df = pd.read_csv("spam.csv", encoding='ISO-8859-1')  # Try 'ISO-8859-1' or 'utf-16'

# ✅ Select only the relevant columns (v1 for labels, v2 for sentences)
df = df[['v1', 'v2']]

# ✅ Inspect the first few rows of the dataframe and the column names
print(df.head())  # This will print the first few rows
print(df.columns)  # This will print the column names

# ✅ Function to translate text to Telugu
def translate_to_telugu(text):
    if pd.isna(text):  # Handle missing values
        return ""

    params = {
        "api-version": "3.0",
        "from": "en",
        "to": "te"  # Change 'hi' to 'te' for Telugu
    }

    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY,
        "Ocp-Apim-Subscription-Region": LOCATION,
        "Content-Type": "application/json"
    }

    body = [{"text": text}]

    response = requests.post(ENDPOINT, params=params, headers=headers, json=body)

    if response.status_code == 200:
        return response.json()[0]["translations"][0]["text"]
    else:
        return f"Error: {response.status_code}"  # Handle API errors

# ✅ Translate all rows in column 'v2' (the sentences) into Telugu
df['v2'] = df['v2'].apply(translate_to_telugu)

# ✅ Save the updated DataFrame (now only Telugu text in v2, with the original labels intact in v1)
df.to_csv("translated_dataset_telugu.csv", index=False, encoding="utf-8-sig")

print("✅ Translation completed! English removed, only Telugu retained in the 'v2' column. File saved as 'translated_dataset_telugu.csv'.")


     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
Index(['v1', 'v2'], dtype='object')
✅ Translation completed! English removed, only Telugu retained in the 'v2' column. File saved as 'translated_dataset_telugu.csv'.
